In [1]:
# Imports
import os

import bokeh
import bokeh.plotting
import numpy as np
import pandas as pd

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# Set path to collect data files
data_path = "../data/raw/"

# Import necessary files
OG_file = os.path.join(data_path, "TARA243.OG.profile.release")

In [3]:
# Read each input data file into pandas
OG_df = pd.read_csv(OG_file, sep='\t')

In [4]:
# Set path to collect data files
data_path = "../data/clean/"

# Import necessary files
metadata_file = os.path.join(data_path, "companion_table_W1.csv")
nutrient_file = os.path.join(data_path, "nutrient_temp_table.csv")
core_file = os.path.join(data_path, "core_annos.csv")

In [5]:
# Read each input data file into pandas
meta_df = pd.read_csv(metadata_file)
cond_df = pd.read_csv(nutrient_file)
core_df = pd.read_csv(core_file)

In [6]:
# Extract sample label, id, and location from metadata
id_df = meta_df.iloc[:,[0, 4, 11, 14, 15, 16]]
id_df = id_df.rename(columns={'Sample label [TARA_station#_environmental-feature_size-fraction]': 'label', 
                              'PANGAEA sample identifier': 'pangea_id',
                              'Marine pelagic biomes (Longhurst 2007)': 'Biome',
                              'Ocean and sea regions (IHO General Sea Areas 1953) [MRGID registered at www.marineregions.com]': 'Region',
                              'Marine pelagic biomes  (Longhurst 2007) [MRGID registered at www.marineregions.com] ': 'Province'
                             })
# Extract temperature and id info for each sample
cond_df = cond_df.iloc[:,[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
cond_df = cond_df.rename(columns={'PANGAEA Sample ID': 'pangea_id', 
                                  'Mean_Lat*': 'Latitude',
                                  'Mean_Long*': 'Longitude',
                                  'Mean_Depth [m]*': 'Depth (m)',
                                  'Mean_Temperature [deg C]*': 'Temperature (°C)',
                                  'Mean_Salinity [PSU]*': 'Salinity (PSU)',
                                  'Mean_Oxygen [umol/kg]*': 'Oxygen (µmol/kg)',
                                  'Mean_Nitrates[umol/L]*': 'Nitrates (µmol/L)',
                                  'NO2 [umol/L]**': 'NO2 (µmol/L)',
                                  'PO4 [umol/L]**': 'PO4 (µmol/L)',
                                 })
# Merge all relevant data into one df
id_df = id_df.merge(cond_df, how='left', on='pangea_id')

In [7]:
OG_df = pd.melt(OG_df, ["cog"], var_name="label", value_name="counts")
OG_df = OG_df.rename(columns={"cog": "OG"})
OG_df.head()

,OG,label,counts
0,sum_not_annotated,TARA_100_DCM_0.22-3,7.617148e+06
1,COG0001,TARA_100_DCM_0.22-3,9.924748e+03
2,COG0002,TARA_100_DCM_0.22-3,9.434435e+03
3,COG0003,TARA_100_DCM_0.22-3,2.337477e+02
4,COG0004,TARA_100_DCM_0.22-3,2.662103e+04


In [8]:
# Add core annotations 
core_df = core_df.rename(columns={"COG": "OG"})
OG_df = OG_df.merge(core_df, how="left", on="OG")

In [9]:
OG_df = OG_df.loc[OG_df["Ocean core"] == False]

In [10]:
abundances_list = []
sample_list = []
OG_list = []
# Calculate relative abundances for each sample
for sample in OG_df.label.unique():
    total = OG_df.loc[OG_df['label'] == sample]['counts'].sum()
    abundances = OG_df.loc[OG_df['label'] == sample].groupby('OG')['counts'].sum()/total
    abundances_list.extend(abundances.values)
    sample_list.extend([sample] * len(abundances))
    OG_list.extend(list(abundances.index))

In [11]:
OG_relative_df = pd.DataFrame({
    'abundance': abundances_list,
    'label': sample_list,
    'OG': OG_list
})

In [12]:
OG_relative_df = OG_relative_df.pivot(index="label", columns=["OG"], values="abundance").fillna(0)

In [13]:
OG_relative_df = OG_relative_df.reset_index()

In [14]:
OG_relative_df = OG_relative_df.merge(id_df, how="left", on="label")

In [15]:
OG_relative_df

,label,COG0478,COG0650,COG0680,COG0833,COG0856,COG1031,COG1039,COG1059,COG1098,...,Province,Latitude,Longitude,Depth (m),Temperature (°C),Salinity (PSU),Oxygen (µmol/kg),Nitrates (µmol/L),NO2 (µmol/L),PO4 (µmol/L)
0,TARA_004_DCM_0.22-1.6,0.000156,0.000000e+00,0.000044,1.883694e-06,0.001772,0.002046,0.000411,0.000027,1.665274e-05,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.57,-6.54,38.7,16.2,36.6,NaN,NaN,NaN,NaN
1,TARA_004_SRF_0.22-1.6,0.000035,0.000000e+00,0.000089,1.814266e-06,0.000665,0.000677,0.000573,0.000037,2.851030e-05,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.55,-6.57,10.0,20.5,36.6,NaN,NaN,NaN,NaN
2,TARA_007_DCM_0.22-1.6,0.000036,0.000000e+00,0.001416,1.577918e-06,0.001400,0.001561,0.000255,0.000072,1.005939e-05,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",37.04,1.95,41.7,17.4,NaN,NaN,NaN,0.00,0.01
3,TARA_007_SRF_0.22-1.6,0.000000,7.556099e-06,0.000546,1.113138e-06,0.000539,0.000613,0.000733,0.000009,2.585322e-06,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",37.02,1.95,7.5,23.8,37.5,NaN,NaN,0.00,0.06
4,TARA_009_DCM_0.22-1.6,0.000077,0.000000e+00,0.000072,2.939981e-07,0.001940,0.001890,0.000214,0.000060,2.216529e-06,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",39.07,5.86,55.0,16.2,37.8,NaN,NaN,0.02,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,TARA_151_DCM_0.22-3,0.000113,0.000000e+00,0.000017,1.175892e-06,0.002065,0.002107,0.000515,0.000116,1.164430e-05,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.19,-28.88,77.6,16.8,36.2,228.5,1.6,0.01,0.01
239,TARA_151_SRF_0.22-3,0.000075,6.278880e-07,0.000020,3.576186e-06,0.003566,0.003932,0.000232,0.000034,0.000000e+00,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.16,-29.01,5.4,17.3,36.2,232.1,0.3,0.02,0.01
240,TARA_152_MES_0.22-3,0.000010,0.000000e+00,0.000079,4.285656e-06,0.000404,0.000644,0.000104,0.000003,1.506727e-07,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,43.74,-16.88,791.8,10.2,35.5,175.3,18.2,0.01,1.12
241,TARA_152_MIX_0.22-3,0.000087,0.000000e+00,0.000019,2.166555e-05,0.002633,0.002960,0.000359,0.000117,7.552205e-05,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,43.79,-16.89,23.9,14.3,36.0,239.4,3.8,0.32,0.17


In [16]:
relative_OG_file = os.path.join(data_path, "relative_OG_with_metadata.csv")
OG_relative_df.to_csv(relative_OG_file, index=False)